# Get Transaction Timestamps

We have the timestamps for insertion and suppression results but they are missing for displacement.

Transactions have no timestamp. Instead, they have a reference to a block, and blocks have timestamp.

In [ ]:
import json
import os
import pickle
import tarfile

from web3 import Web3

In [ ]:
data_directory = os.path.join("..", "..", "..", "data")

Uncomment the line for the right web3 provider (websocket or http) and fill the host and port.

In [ ]:
# server = Web3(Web3.WebsocketProvider("ws://host:port"))

# server = Web3(Web3.HTTPProvider("http://host:port"))

Decompress the results file (if needed).

In [ ]:
results_file_path = os.path.join(data_directory, "displacement_results.json")

if not os.path.exists(results_file_path):
    with tarfile.open(os.path.join(data_directory, "displacement_results.tar.xz"), "r:xz", encoding="utf-8") as compressed_file:
        compressed_file.extract("displacement_results.json", data_directory)

Read the results one line at a time. Each line is json encoded.

In [ ]:
with open(results_file_path, "r", encoding="utf-8") as results_file:
    results = []
    line = results_file.readline().strip()
    while line != "":
        results.append(json.loads(line))
        line = results_file.readline().strip()

Fetch the timestamp for the block.

In [ ]:
timestamp_by_block = {}

In [ ]:
def update_block_timestamps(block_number):
    if block_number not in timestamp_by_block:
        timestamp_by_block[block_number] = server.eth.getBlock(block_number)["timestamp"]

for result in results:
    update_block_timestamps(result["attacker_transaction"]["blockNumber"])
    update_block_timestamps(result["victim_transaction"]["blockNumber"])

In [ ]:
with open(os.path.join(data_directory, "displacement_block_timestamps.pickle"), "wb") as pikle_file:
    pickle.dump(timestamp_by_block, pikle_file)